# Introdução a Análise Estatística em Python

Análises estatísticas são fundamentais em qualquer tarefa de Ciência de Dados. Neste notebook apresentamos conceitos básicos do uso da biblioteca [StatsModels](https://www.statsmodels.org), que implementa uma grande gama de modelos estatísticos.

In [1]:
# Importação de pacotes
import pandas as pd

%matplotlib inline

## Lendo dados de um arquivo

O DataFrame carregado contém dados de apartamentos para alugar na cidade de Curitiba.

In [3]:
# lê o arquivo CSV
df = pd.read_csv('../data/aluguel.csv')
# mostra o conteúdo do DataFrame
df.head(3)

,codigo,endereco,quartos,suite,area,vaga,aluguel,condominio,data
0,34,Rua Desembargador Westphalen,2,0,90,0,900,371,11/10/17
1,167,Rua Jose Loureiro,2,0,64,0,650,428,15/07/17
2,6784,Rua Jose Loureiro,2,0,81,0,1100,400,23/08/17


## Comparando médias

Frequentemente precisamos medir e comparar médias de subconjuntos dos nossos dados. Por exemplo, podemos querer medir a média dos valores dos aluguéis para se ter uma ideia do custo de vida na cidade de Curitiba. 

Porém, para sabermos com certeza o valor da média dos aluguéis, precisaríamos ter **todos** os valores de todos os aluguéis das cidade. Como obter todos os valores é quase sempre impossível, fazemos uma amostragem e usamos a média da amostragem como sendo uma média estimada dos valores.

No nosso caso, a amostra são os 20 apartamentos presentes no DataFrame. Como temos poucos dados, é de se esperar que eles nos dão pouca certeza sobre o verdadeiro valor da média. Uma forma de calcular esta certeza é usando intervalos de confiança. O exemplo abaixo calcula a média dos aluguéis e respectivo intervalo de confiança. O intervalo indica que a média real dos aluguéis (considerando que nossa amostragem foi aleatória, o que não é verdade), deve ser entre 745 e 1050.

In [4]:
import statsmodels.stats.api as sms

media = df['aluguel'].mean()

intervalo = sms.DescrStatsW(df['aluguel']).tconfint_mean()

print("Média: ", media, "Intervalo: ", intervalo)

Média:  898.0 Intervalo:  (745.3791042764885, 1050.6208957235115)


Podemos também querer saber se existe diferença significativa entre as médias dos valores de aluguéis de 1 e 2 quartos. No exemplo abaixo criamos dois dataframes para cada quantidade de quartos e usamos o método `CompareMeans` para obter estatísticas sobre a diferença dos valores das médias. O resultado indica que, com 95% de certeza, podemos afirmar que a diferença entre as médias é entre -641 e -146. Como temos uma boa certeza que a diferença não é zero, podemos afirmar que aluguéis de 2 quartos são mais caros que os de 1.

In [8]:
df_1q = df[df['quartos']==1]
df_2q = df[df['quartos']==2]

cm = sms.CompareMeans(sms.DescrStatsW(df_1q['aluguel']), sms.DescrStatsW(df_2q['aluguel']))
cm.summary()

,coef,std err,t,P>|t|,[0.025,0.975]
subset #1,-394.0000,117.584,-3.351,0.004,-641.035,-146.965


Já para a diferença entre médias de condomínios, não podemos afirmar com certeza que condomínios de apartamentos de 2 quartos são mais caros do que os de 1. Veja que a diferença entre as médias pode ser 0, com isso não temos evidências suficientes para dizer que as médias são diferentes. Isto tambeḿ é mostrado pelo alto valor de *P* (0.36).

In [17]:
cm = sms.CompareMeans(sms.DescrStatsW(df_1q['condominio']), sms.DescrStatsW(df_2q['condominio']))
cm.summary()

,coef,std err,t,P>|t|,[0.025,0.975]
subset #1,-38.5000,41.387,-0.930,0.365,-125.451,48.451


## Regressão Linear


Modelos de regressão são muito importantes em Ciência de Dados. Eles são usados para se entender a influência das variáveis sobre um determinado aspecto ou para prever o comportamento deste aspecto. Por exemplo, podemos querer entender como as variáveis `quartos, area, vaga` influenciam no valor do aluguel. 

Abaixo construímos um modelo de regressão linear. O sumário apresentado mostra que o modelo captura relativamente bem a variância no nosso problema (R-squared = 0.62). Das variáveis incluídas, apenas a área se mostrou confiável para explicar o valor do aluguel (P = 0.02). 

Este é um modelo simples e estamos desconsiderando diversas questões (como covariância entre as variáveis). 

In [9]:
from statsmodels.formula.api import ols

model = ols("aluguel ~ quartos + area + vaga", data=df)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                aluguel   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     8.933
Date:                Tue, 17 Sep 2019   Prob (F-statistic):            0.00104
Time:                        15:05:46   Log-Likelihood:                -133.77
No. Observations:                  20   AIC:                             275.5
Df Residuals:                      16   BIC:                             279.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    104.7992    166.560      0.629      0.538    -248.293     457.891
quartos      173.0788    138.969      1.245      0.231    -121.522     467.680
area           8.4528      3.448      2.452      0.026       1.144      15.762
vaga         -36.8360    144.643     -0.255      0.802    -343.466     269.794
==============================================================================
Omnibus:                        2.313   Durbin-Watson:                   1.468
Prob(Omnibus):                  0.315   Jarque-Bera (JB):                1.085
Skew:                           0.552   Prob(JB):                        0.581
Kurtosis:                       3.288   Cond. No.                         279.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Usando o modelo contruído acima, podemos prever o valor do aluguel para um apartamento qualquer. Abaixo especificamos os dados de um novo apartamento e o modelo previu que seu aluguel deve custar 921.

In [45]:
novo_apartamento = pd.DataFrame([{'quartos': 2, 'area': 60, 'vaga': 1}])
response.predict(novo_apartamento)

0    921.290363
dtype: float64